In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import cv2

#processing
from scipy.signal import savgol_filter
from scipy import signal
from brain_observatory_analysis.behavior.video_qc import annotation_tools
from brain_observatory_qc.data_access import from_lims_utilities
from brain_observatory_analysis.ophys.experiment_loading import start_lamf_analysis

C:\Users\iryna.yavorska\Anaconda3\envs\mFish_glm\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

#### load file with neuropixel paths

In [3]:
filename = 'vbn_video_paths.xlsx'
vbn_video_paths = pd.read_excel(filename)
vbn_video_paths.keys()

Index(['experiment_id', 'face_video', 'face_video_metadata', 'face_dlc_output',
       'side_video', 'side_video_metadata', 'side_dlc_output', 'eye_video',
       'eye_video_metadata', 'eye_dlc_output'],
      dtype='object')

In [4]:
### This table connects specimen ids in prod0 folder to mouse ids.
### Clark will add a function to brain_observatory_qc that can do that without this table
base_dir = '//allen/programs/mindscope/workgroups/learning/behavioral_video_plots'
mouse_table = pd.read_csv(os.path.join(base_dir, '20230713_SS_LAMFdirectories.csv'))


#### learning mFish ids

In [5]:
experiment_table = start_lamf_analysis()
np.unique(experiment_table.donor_id.apply(int))

Should be on older version of allensdk branch, let MJD know if not


c:\users\iryna.yavorska\documents\github\allensdk_mjd\allensdk\allensdk\brain_observatory\behavior\behavior_project_cache\tables\util\prior_exposure_processing.py:165: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  counts = df.groupby(['mouse_id'])['to'].apply(cumsum)


array([1142290477, 1148010756, 1155072208, 1172320751, 1179706234,
       1187453999, 1194433355, 1199272999, 1216869852], dtype=int64)

#### paths to example ophys videos and output

In [62]:
session = 'videos'

folder = 'face_tracking'#, 'side_tracking', 'eye_tracking',

input_path = 'E:/face_video_annotation-iryna-2023-08-28/videos'

dlc_path = 'E:/face_video_annotation-iryna-2023-08-28/previous_dlc'


In [63]:
dlc_files = annotation_tools.get_h5file(dlc_path )
dlc_files = np.sort(dlc_files)

found 6 h5 files


In [64]:
output_path =  '//allen/programs/mindscope/workgroups/learning/behavioral_video_annotation/NCB_plots/face'

In [65]:
all_DLC = []
for dlc_file in dlc_files:
    all_DLC.append(annotation_tools.read_DLC_h5file(dlc_file))

loaded DLC data
loaded DLC data
loaded DLC data
loaded DLC data
loaded DLC data
loaded DLC data


#### load movie and plot tracked points

In [66]:
mp4files = annotation_tools.get_mp4file(input_path)
mp4files = np.sort(mp4files)

camera view was not provided, returning all mp4 files
mp4 file: ['E:/face_video_annotation-iryna-2023-08-28/videos\\1170569654_Face_20220413T101321.mp4', 'E:/face_video_annotation-iryna-2023-08-28/videos\\1188450756_Face_20220701T090612.mp4', 'E:/face_video_annotation-iryna-2023-08-28/videos\\1193461885_Face_20220722T084813.mp4', 'E:/face_video_annotation-iryna-2023-08-28/videos\\1224566242_Face_20221108T085010.mp4', 'E:/face_video_annotation-iryna-2023-08-28/videos\\1226957088_Face_20221118T091700.mp4', 'E:/face_video_annotation-iryna-2023-08-28/videos\\1229836719_Face_20221201T090953.mp4', 'E:/face_video_annotation-iryna-2023-08-28/videos\\1231992811_Face_20221210T113121.mp4']


In [67]:
dlc_files

array(['E:/face_video_annotation-iryna-2023-08-28/previous_dlc\\1188450756_Face_20220701T090612DeepCut_resnetNone_np3_face_camMar30shuffle1_1030000.h5',
       'E:/face_video_annotation-iryna-2023-08-28/previous_dlc\\1193461885_Face_20220722T084813DeepCut_resnetNone_np3_face_camMar30shuffle1_1030000.h5',
       'E:/face_video_annotation-iryna-2023-08-28/previous_dlc\\1224566242_Face_20221108T085010DeepCut_resnetNone_np3_face_camMar30shuffle1_1030000.h5',
       'E:/face_video_annotation-iryna-2023-08-28/previous_dlc\\1226957088_Face_20221118T091700DeepCut_resnetNone_np3_face_camMar30shuffle1_1030000.h5',
       'E:/face_video_annotation-iryna-2023-08-28/previous_dlc\\1229836719_Face_20221201T090953DeepCut_resnetNone_np3_face_camMar30shuffle1_1030000.h5',
       'E:/face_video_annotation-iryna-2023-08-28/previous_dlc\\1231992811_Face_20221210T113121DeepCut_resnetNone_np3_face_camMar30shuffle1_1030000.h5'],
      dtype='<U141')

## example use of video processing

In [53]:
bodyparts = ['whisker_pad_r', 'whisker_pad_l', 'nose_tip', 'tongue',
       'paw_forward_l', 'wrist_l', 'paw_forward_lh', 'wrist_lh',
       'paw_forward_r', 'wrist_r', 'paw_forward_rh', 'wrist_rh',
       'tail_mid', 'tail_base', 'spout_tip']

In [68]:
%%time
for i,m in enumerate(mp4files):
    dlc =  all_DLC[i]#[all_DLC[i].bodyparts.isin(bodyparts)]
    annotation_tools.process_video(output_path, mp4files[i+1], dlc)

Processing video E:/face_video_annotation-iryna-2023-08-28/videos\1188450756_Face_20220701T090612.mp4 with 60.0 fps at 658x492...
found #220487 frames to plot


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [06:05<00:00,  2.73it/s]


Video processing complete.
Processing video E:/face_video_annotation-iryna-2023-08-28/videos\1193461885_Face_20220722T084813.mp4 with 60.0 fps at 658x492...
found #273619 frames to plot


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [06:10<00:00,  2.70it/s]


Video processing complete.
Processing video E:/face_video_annotation-iryna-2023-08-28/videos\1224566242_Face_20221108T085010.mp4 with 30.0 fps at 658x492...
found #113852 frames to plot


...

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [06:57<00:00,  2.40it/s]


Video processing complete.
Processing video E:/face_video_annotation-iryna-2023-08-28/videos\1231992811_Face_20221210T113121.mp4 with 30.0 fps at 658x492...
found #113616 frames to plot


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [06:24<00:00,  2.60it/s]

Video processing complete.


IndexError: list index out of range

In [46]:
experiment_table.index

Int64Index([1153099568, 1153099558, 1153099564, 1153099565, 1153099562,
            1153099555, 1153099561, 1153099559, 1153662768, 1153662777,
            ...
            1231598631, 1231598628, 1232481420, 1232481421, 1232481433,
            1232481418, 1232481426, 1232481428, 1232481429, 1232481423],
           dtype='int64', name='ophys_experiment_id', length=1160)

#### get_general_info_for_LIMS_imaging_id

In [ ]:
ophys_experiment_id = 1153099561
general_info = from_lims_utilities.get_general_info_for_LIMS_imaging_id("ophys_experiment_id", ophys_experiment_id)

In [ ]:
general_info.keys()

Index(['ophys_experiment_id', 'ophys_session_id', 'behavior_session_id',
       'foraging_id', 'ophys_container_id', 'supercontainer_id',
       'experiment_workflow_state', 'session_workflow_state',
       'container_workflow_state', 'specimen_id', 'donor_id', 'specimen_name',
       'date_of_acquisition', 'session_type', 'targeted_structure', 'depth',
       'equipment_name', 'project', 'experiment_storage_directory',
       'behavior_storage_directory', 'session_storage_directory',
       'container_storage_directory', 'supercontainer_storage_directory',
       'specimen_storage_directory'],
      dtype='object')

## example use of frame selection

In [60]:
cam_json= annotation_tools.get_jsonfile(input_path, folder)
type(cam_json)

json file: //allen/programs/mindscope/production/learning/prod0/specimen_1153540225\ophys_session_1165583926\1165583926_Face_20220321T092804.json


str

In [53]:
syncfile = annotation_tools.get_syncfile(input_path)

sync file: //allen/programs/mindscope/production/learning/prod0/specimen_1153540225\ophys_session_1165583926\1165583926_20220321T9283.h5


In [55]:
sync_dataobject = annotation_tools.get_sync_dataset(syncfile)

sync dataset: <allensdk.brain_observatory.sync_dataset.Dataset object at 0x000002134C65A670>


In [57]:
cam_json

'//allen/programs/mindscope/production/learning/prod0/specimen_1153540225\\ophys_session_1165583926\\1165583926_Face_20220321T092804.json'

In [64]:
frame_times = annotation_tools.get_cam_timestamps(sync_dataobject, cam_json, account_for_metadata_frame=True)

In [65]:
## 
frame_times

array([           nan, 3.02490000e-01, 3.19150000e-01, ...,
       4.80008142e+03, 4.80009809e+03, 4.80011475e+03])

#### Use timestamps to get the same frames from a different video

In [72]:
output_path

'//allen/programs/mindscope/workgroups/learning/behavioral_video_plots\\specimen_1153540225\\ophys_session_1165583926\\face_tracking'

In [70]:
# LOAD FAME FRAMEs
face_frame_dir = r'\\allen\programs\mindscope\workgroups\learning\behavioral_video_annotation\specimen_1148010767\ophys_session_1158851430_face-iryna-2023-07-27\labeled-data\1158851430_Face_20220217T111922'

face_frames = glob.glob(face_frame_dir + '/img*')

created_times = [os.path.getctime(f) for f in face_frames]

# get frame numbers from file names
frame_no = [annotation_tools.get_frame_index(file) for file in face_frames]

# get time stampa of each frame
selected_timestamps = frame_times[frame_no]

In [71]:
selected_timestamps 

array([ 338.04517,  393.16064,  449.27609,  628.03887, 1141.77774,
       1192.52664, 1219.22606, 1264.59175, 1268.67499, 1308.82412,
       1405.7887 , 1525.6861 , 1632.33379, 1719.63189, 1777.63062,
       1785.56378, 1972.8597 , 2035.625  , 2370.38437, 2416.2167 ,
       2439.18287, 2485.58186, 2515.26454, 2535.23078, 2541.34731,
       2585.04635, 2667.51122, 2870.49013, 3011.52038, 3200.49959,
       3230.84893, 3280.16452, 3305.76396, 3321.71362, 3342.71316,
       3435.69446, 3521.72592, 3550.27529, 3578.27468, 3612.95726])

In [ ]:
# get matching side view timestamps
side_path = path_file[path_file['full_exp_id'] == experiment_id]['side_timestamp_path']
side_time_stamps = np.load(side_path.values[0])


side_frame_no = [np.nanargmin(np.abs(side_time_stamps - t)) for t in frame_time_stamps]
side_matching_time_stamps = side_time_stamps[side_frame_no]

max_time_diff_between_views = np.max(side_matching_time_stamps - frame_time_stamps)
assert(max_time_diff_between_views<0.016)